# Calculating features

In [1]:
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input

import numpy as np
import pandas as pd
from os.path import join
import os

Using TensorFlow backend.


In [2]:
regions = ['borde_rural', 'borde_soacha', 'mixco_1_and_ebenezer', 'mixco_3']

In [3]:
region = 'dennery'
train_dir = join('..', '..', 'data', 'data2', region, 'roofs_train')
test_dir = join('..', '..', 'data', 'data2', region, 'roofs_test')
materials = {'concrete_cement':0, 'healthy_metal':1, 'incomplete':2, 'irregular_metal':3, 'other':4}

Use a pretrained network to compute features from the images, which are later classified. Import the model from the Kerase library and remove the top layer by setting `include_top=False`. The last layer is pooled such that we get outputs of size 1x2048 instead of 7x7x2048. The pooling method `'max'` is chosen empirically.

In [5]:
model = ResNet50(weights='imagenet', include_top=False, pooling='max')
model.summary()













94658560/94653016 [==============================] - 5s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
________________________________

## 1. Calculate features on the training data

The preprocessed data is stored in such a way, that every region has their own folder, that contains a subfolder defined as `train_dir` above. This folder again is subdivided by material. Images carry their `id` as filename.

Traverse all images in their corresponding folders which indicate the material and use `model.predict()` to get the features.

In [6]:
def calculate_features_train(train_dir):
    
    # Initialize a dataframe for the features    
    columns = ['id', 'features', 'label']
    df_features = pd.DataFrame(columns=columns)
    
    data = []

    # Walk through all images
    for material in materials.keys():
        material_fp = join(train_dir, material)
        for root, dirs, files in os.walk(material_fp):
            for file in files:
                img_fp = join(material_fp, file)
                label = materials[material]
                id = file.split('.')[0]
                print(id, "labeled as", material, ":", label)

                # Pad if size is too small, preprocess
                img = image.load_img(img_fp, target_size=(224, 224))
                img_data = image.img_to_array(img)
                img_data = np.expand_dims(img_data, axis=0)
                img_data = preprocess_input(img_data)

                # Compute features 
                features = model.predict(img_data)
                features_np = np.array(features).flatten()

                data.append({'id': id ,
                             'features': features_np,
                             'label': label})
                #df_features = df_features.append({'id': id ,
                #                                  'features': resnet50_feature_np,
                #                                  'label': label}, ignore_index=True)
    
    df_features = pd.DataFrame(data)
    return df_features

The results are stored in a dataframe as shown below and saved to disk as a pickle file.

In [7]:
print(train_dir)

../../data/data2/borde_soacha/roofs_train


In [8]:
features_train = calculate_features_train(train_dir)

7a20f7ac labeled as concrete_cement : 0
7a38b298 labeled as concrete_cement : 0
7a3a662e labeled as concrete_cement : 0
7a3d7d28 labeled as concrete_cement : 0
7a397142 labeled as concrete_cement : 0
7a382b34 labeled as concrete_cement : 0
7a38f03c labeled as concrete_cement : 0
7a238a3a labeled as concrete_cement : 0
7a2888fa labeled as concrete_cement : 0
7a3709c0 labeled as concrete_cement : 0
7a1d5caa labeled as concrete_cement : 0
7a1d085e labeled as concrete_cement : 0
7a2eb22a labeled as concrete_cement : 0
7a23f1dc labeled as concrete_cement : 0
7a3c5e66 labeled as concrete_cement : 0
7a2ac9b2 labeled as concrete_cement : 0
7a2bc9de labeled as concrete_cement : 0
7a2ad164 labeled as concrete_cement : 0
7a2792a6 labeled as concrete_cement : 0
7a2eea10 labeled as concrete_cement : 0
7a372f54 labeled as concrete_cement : 0
7a2ae12c labeled as concrete_cement : 0
7a3b557a labeled as concrete_cement : 0
7a3b9904 labeled as concrete_cement : 0
7a1d64a2 labeled as concrete_cement : 0


7a1e9192 labeled as concrete_cement : 0
7a2bac6a labeled as concrete_cement : 0
7a449e5a labeled as concrete_cement : 0
7a239a66 labeled as concrete_cement : 0
7a1e7158 labeled as concrete_cement : 0
7a25838a labeled as concrete_cement : 0
7a2bb426 labeled as concrete_cement : 0
7a2abc10 labeled as concrete_cement : 0
7a24b78e labeled as concrete_cement : 0
7a1ca080 labeled as concrete_cement : 0
7a200b44 labeled as concrete_cement : 0
7a21ce8e labeled as concrete_cement : 0
7a2f63be labeled as concrete_cement : 0
7a4072d0 labeled as concrete_cement : 0
7a4097f6 labeled as concrete_cement : 0
7a1c7646 labeled as concrete_cement : 0
7a231956 labeled as concrete_cement : 0
7a2035ce labeled as concrete_cement : 0
7a310d72 labeled as concrete_cement : 0
7a285128 labeled as concrete_cement : 0
7a1cd0a0 labeled as concrete_cement : 0
7a40aa70 labeled as concrete_cement : 0
7a319440 labeled as concrete_cement : 0
7a36181c labeled as concrete_cement : 0
7a44a2ba labeled as concrete_cement : 0


7a2aa810 labeled as healthy_metal : 1
7a38cb48 labeled as healthy_metal : 1
7a365e44 labeled as healthy_metal : 1
7a31ef58 labeled as healthy_metal : 1
7a219f4a labeled as healthy_metal : 1
7a44abfc labeled as healthy_metal : 1
7a41660e labeled as healthy_metal : 1
7a44493c labeled as healthy_metal : 1
7a1fb7c0 labeled as healthy_metal : 1
7a3132e8 labeled as healthy_metal : 1
7a3c13a2 labeled as healthy_metal : 1
7a2cc6ae labeled as healthy_metal : 1
7a1e5d6c labeled as healthy_metal : 1
7a3f4ec8 labeled as healthy_metal : 1
7a1cf40e labeled as healthy_metal : 1
7a2f4564 labeled as healthy_metal : 1
7a354d88 labeled as healthy_metal : 1
7a4256b8 labeled as healthy_metal : 1
7a1ef060 labeled as healthy_metal : 1
7a39ed7a labeled as healthy_metal : 1
7a326186 labeled as healthy_metal : 1
7a34d6be labeled as healthy_metal : 1
7a282a54 labeled as healthy_metal : 1
7a1f7c74 labeled as healthy_metal : 1
7a39eca8 labeled as healthy_metal : 1
7a1e8ab2 labeled as healthy_metal : 1
7a3cd0f8 lab

7a2be7de labeled as healthy_metal : 1
7a3a03c8 labeled as healthy_metal : 1
7a3a149e labeled as healthy_metal : 1
7a396666 labeled as healthy_metal : 1
7a2ad600 labeled as healthy_metal : 1
7a2f6e18 labeled as healthy_metal : 1
7a288b16 labeled as healthy_metal : 1
7a378ac6 labeled as healthy_metal : 1
7a22ebd4 labeled as healthy_metal : 1
7a2632b2 labeled as healthy_metal : 1
7a2de89a labeled as healthy_metal : 1
7a321118 labeled as healthy_metal : 1
7a241040 labeled as healthy_metal : 1
7a3015b6 labeled as healthy_metal : 1
7a34c43a labeled as healthy_metal : 1
7a3de07e labeled as healthy_metal : 1
7a32badc labeled as healthy_metal : 1
7a3226ee labeled as healthy_metal : 1
7a3e764c labeled as healthy_metal : 1
7a3dfeba labeled as healthy_metal : 1
7a273360 labeled as healthy_metal : 1
7a37a93e labeled as healthy_metal : 1
7a3a2d30 labeled as healthy_metal : 1
7a3956b2 labeled as healthy_metal : 1
7a3d622a labeled as healthy_metal : 1
7a2672c2 labeled as healthy_metal : 1
7a3d6158 lab

7a321dc0 labeled as healthy_metal : 1
7a42bbb2 labeled as healthy_metal : 1
7a40f4bc labeled as healthy_metal : 1
7a2e3fd4 labeled as healthy_metal : 1
7a32240a labeled as healthy_metal : 1
7a350a62 labeled as healthy_metal : 1
7a42749a labeled as healthy_metal : 1
7a4281a6 labeled as healthy_metal : 1
7a1eaa92 labeled as healthy_metal : 1
7a242b84 labeled as healthy_metal : 1
7a25c034 labeled as healthy_metal : 1
7a26c90c labeled as healthy_metal : 1
7a3936dc labeled as healthy_metal : 1
7a22dd10 labeled as healthy_metal : 1
7a2dfde4 labeled as healthy_metal : 1
7a3a1886 labeled as healthy_metal : 1
7a2fccbe labeled as healthy_metal : 1
7a26f31e labeled as healthy_metal : 1
7a36ada4 labeled as healthy_metal : 1
7a27e4f4 labeled as healthy_metal : 1
7a229710 labeled as healthy_metal : 1
7a271808 labeled as healthy_metal : 1
7a352cfe labeled as healthy_metal : 1
7a34103a labeled as healthy_metal : 1
7a3fb598 labeled as healthy_metal : 1
7a41fcfe labeled as healthy_metal : 1
7a32f0b0 lab

...

7a38837c labeled as irregular_metal : 3
7a370d76 labeled as irregular_metal : 3
7a3391b4 labeled as irregular_metal : 3
7a32923c labeled as irregular_metal : 3
7a3eaefa labeled as irregular_metal : 3
7a2715f6 labeled as irregular_metal : 3
7a42c490 labeled as irregular_metal : 3
7a38e3bc labeled as irregular_metal : 3
7a3840f6 labeled as irregular_metal : 3
7a44079c labeled as irregular_metal : 3
7a395900 labeled as irregular_metal : 3
7a1cfa08 labeled as irregular_metal : 3
7a321d52 labeled as irregular_metal : 3
7a1d6af6 labeled as irregular_metal : 3
7a33b59a labeled as irregular_metal : 3
7a33fe2e labeled as irregular_metal : 3
7a4090ee labeled as irregular_metal : 3
7a3c8ed6 labeled as irregular_metal : 3
7a222898 labeled as irregular_metal : 3
7a24fece labeled as irregular_metal : 3
7a3c6582 labeled as irregular_metal : 3
7a3b19f2 labeled as irregular_metal : 3
7a2c22ee labeled as irregular_metal : 3
7a3771e4 labeled as irregular_metal : 3
7a41df12 labeled as irregular_metal : 3


7a22f07a labeled as irregular_metal : 3
7a21fb20 labeled as irregular_metal : 3
7a1d9ec2 labeled as irregular_metal : 3
7a278324 labeled as irregular_metal : 3
7a20363c labeled as irregular_metal : 3
7a2f5522 labeled as irregular_metal : 3
7a2af572 labeled as irregular_metal : 3
7a3d051e labeled as irregular_metal : 3
7a39b3be labeled as irregular_metal : 3
7a1d1b64 labeled as irregular_metal : 3
7a3f833e labeled as irregular_metal : 3
7a3ba64c labeled as irregular_metal : 3
7a25441a labeled as irregular_metal : 3
7a3717bc labeled as irregular_metal : 3
7a3de5c4 labeled as irregular_metal : 3
7a42aa8c labeled as irregular_metal : 3
7a2b43e2 labeled as irregular_metal : 3
7a312c58 labeled as irregular_metal : 3
7a3d3d9a labeled as irregular_metal : 3
7a207246 labeled as irregular_metal : 3
7a2edfca labeled as irregular_metal : 3
7a383034 labeled as irregular_metal : 3
7a3797fa labeled as irregular_metal : 3
7a37709a labeled as irregular_metal : 3
7a21d294 labeled as irregular_metal : 3


7a24da0c labeled as irregular_metal : 3
7a2d3472 labeled as irregular_metal : 3
7a236fdc labeled as irregular_metal : 3
7a35e87e labeled as irregular_metal : 3
7a1e9d7c labeled as irregular_metal : 3
7a224d82 labeled as irregular_metal : 3
7a2e1afe labeled as irregular_metal : 3
7a1cd8a2 labeled as irregular_metal : 3
7a1ec3ec labeled as irregular_metal : 3
7a258894 labeled as irregular_metal : 3
7a3b6966 labeled as irregular_metal : 3
7a4201ea labeled as irregular_metal : 3
7a39e58c labeled as irregular_metal : 3
7a2f3e52 labeled as irregular_metal : 3
7a1e3792 labeled as irregular_metal : 3
7a36fade labeled as irregular_metal : 3
7a28d350 labeled as irregular_metal : 3
7a2c5354 labeled as irregular_metal : 3
7a222406 labeled as irregular_metal : 3
7a1c89ba labeled as irregular_metal : 3
7a30d924 labeled as irregular_metal : 3
7a27e95e labeled as irregular_metal : 3
7a3f64e4 labeled as irregular_metal : 3
7a21f6ac labeled as irregular_metal : 3
7a31b13c labeled as irregular_metal : 3


7a2e8c6e labeled as irregular_metal : 3
7a333f7a labeled as irregular_metal : 3
7a34a2e8 labeled as irregular_metal : 3
7a348920 labeled as irregular_metal : 3
7a2ebd38 labeled as irregular_metal : 3
7a412d60 labeled as irregular_metal : 3
7a43aafe labeled as irregular_metal : 3
7a40d3f6 labeled as irregular_metal : 3
7a2542da labeled as irregular_metal : 3
7a364440 labeled as irregular_metal : 3
7a2560d0 labeled as irregular_metal : 3
7a3e950a labeled as irregular_metal : 3
7a36a4d0 labeled as irregular_metal : 3
7a41257c labeled as irregular_metal : 3
7a30a080 labeled as irregular_metal : 3
7a43f1a8 labeled as irregular_metal : 3
7a29d8f4 labeled as irregular_metal : 3
7a2ae26c labeled as irregular_metal : 3
7a3c009c labeled as irregular_metal : 3
7a24d890 labeled as irregular_metal : 3
7a40ead0 labeled as irregular_metal : 3
7a424b46 labeled as irregular_metal : 3
7a374930 labeled as irregular_metal : 3
7a25e172 labeled as irregular_metal : 3
7a27d022 labeled as irregular_metal : 3


7a1ee80e labeled as irregular_metal : 3
7a41cd6a labeled as irregular_metal : 3
7a21cf60 labeled as irregular_metal : 3
7a2a0432 labeled as irregular_metal : 3
7a2e64c8 labeled as irregular_metal : 3
7a37a790 labeled as irregular_metal : 3
7a3e1b8e labeled as irregular_metal : 3
7a29b400 labeled as irregular_metal : 3
7a3849a2 labeled as irregular_metal : 3
7a36d5e0 labeled as irregular_metal : 3
7a3a824e labeled as irregular_metal : 3
7a35102a labeled as irregular_metal : 3
7a285efc labeled as irregular_metal : 3
7a3b07b4 labeled as irregular_metal : 3
7a20d68c labeled as irregular_metal : 3
7a360944 labeled as other : 4
7a307a06 labeled as other : 4
7a3ec1f6 labeled as other : 4
7a3af35a labeled as other : 4
7a2e2be8 labeled as other : 4
7a220b06 labeled as other : 4
7a3f71c8 labeled as other : 4
7a319b84 labeled as other : 4
7a3e5342 labeled as other : 4
7a209b68 labeled as other : 4
7a1ccef2 labeled as other : 4
7a3f2628 labeled as other : 4
7a34c0e8 labeled as other : 4
7a277d70 l

In [9]:
features_train.head()

,id,features,label
0,7a20f7ac,"[4.636075, 2.3204336, 1.4748516, -0.0, -0.0, 1...",0
1,7a38b298,"[2.326549, 3.0096893, 0.1731861, 1.4067602, 0....",0
2,7a3a662e,"[2.7476623, 1.8925416, -0.0, 0.87210643, -0.0,...",0
3,7a3d7d28,"[6.3516073, 3.722457, 0.3522606, -0.0, -0.0, 4...",0
4,7a397142,"[0.38750732, 0.010527623, -0.0, 5.6719933, 7.7...",0


In [10]:
import pickle
from os import makedirs
from os.path import exists

pickle_path = join('..', '..', 'pickles')
if not exists(pickle_path):
    makedirs(pickle_path)

with open(join(pickle_path, 'resnet50_features_' + region + '_train.pkl' ), 'wb') as f:
    pickle.dump(features_train, f)

### Visualize the features

Reduce the dimensionality to 2D using tSNE (https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib
from sklearn.decomposition import PCA
%matplotlib inline

In [ ]:
def plot_tSNE(features, labels=None, number_of_materials=5):
    if labels is None:
        labels = np.zeros((features.shape[0]))

    # Visualization_
    #pca_object = PCA(n_components=50)
    #pca_features = pca_object.fit_transform(features)
    tsne_features = TSNE(n_components=2).fit_transform(features)

    # define the colormap
    cmap = plt.cm.jet
    # extract all colors from the .jet map
    cmaplist = [cmap(i) for i in range(cmap.N)]
    # create the new map
    cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)

    # define the bins and normalize
    bounds = np.linspace(0, number_of_materials, number_of_materials + 1)
    norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
    
    print(tsne_features.shape)

    plt.figure()
    scat = plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=labels, cmap=cmap, norm=norm)
    cb = plt.colorbar(scat, spacing='proportional', ticks=bounds)

Convert features within the dataframe to a single matrix

In [ ]:
feat_matrix = features_train['features'].to_numpy()
feat_matrix = np.column_stack(feat_matrix).transpose()

labels = features_train['label'].to_numpy()

In [ ]:
plot_tSNE(feat_matrix, labels)

## 2. Calculate features on the test data


This procedure is similar as for the training data, except that we do not know the labels and the folder with test images is not subdivided by material

In [ ]:
def calculate_features_test(test_dir):
    
    # Initialize a dataframe for the features    
    columns = ['id', 'features']
    df_features = pd.DataFrame(columns=columns)

    # Walk through all images
    for root, dirs, files in os.walk(test_dir):
        for file in files:
            img_fp = join(root, file)
            id = file.split('.')[0]
            print("Calculate features for", id)

            # Pad if size is too small, preprocess
            img = image.load_img(img_fp, target_size=(224, 224))
            img_data = image.img_to_array(img)
            img_data = np.expand_dims(img_data, axis=0)
            img_data = preprocess_input(img_data)

            # Compute features 
            features = model.predict(img_data)
            features_np = np.array(features).flatten()

            df_features = df_features.append({'id': id ,
                                              'features': features_np},
                                             ignore_index=True)

    return df_features

In [ ]:
features_test = calculate_features_test(test_dir)

In [ ]:
features_test.head()

In [ ]:
with open(join(pickle_path, 'inceptionv3_features_' + region + '_test.pkl' ), 'wb') as f:
    pickle.dump(features_test, f)